![Logo](https://github.com/statistical-thinking/ki-enna-demo/blob/main/content/img/ki-enna-logo.png?raw=true)
## ⚙️ - Anleitung
Mit **KI-ENNA** ([Homepage](https://www.statistical-thinking.de/ki-enna.html)), (E)inem (N)euronalen (N)etz zum (A)usprobieren, kann in wenigen Schritten ein eigenes **neuronales Netzwerk** ([Wiki](https://en.wikipedia.org/wiki/Neural_network_(machine_learning))) aus dem Kontext des **Deep Learnings** ([Wiki](https://en.wikipedia.org/wiki/Deep_learning)) und als methodische Grundlage der **Künstlichen Intelligenz** ([Wiki](https://en.wikipedia.org/wiki/Artificial_intelligence)) generiert, trainiert und evaluiert werden. Die Bedienung erfolgt über ein interaktives **Jupyter Notebook** ([Wiki](https://en.wikipedia.org/wiki/Project_Jupyter#Jupyter_Notebook)), bei dem die entsprechenden Codes in **Python** ([Wiki](https://en.wikipedia.org/wiki/Python_(programming_language))) nacheinander ausgeführt werden können. Die Codes können über *View / Expand* eingeblendet und *View / Collapse* ausgeblendet sowie über *Run* ausgeführt werden.

## 📤 - Datensatz
Im ersten Schritt kann ein **CSV** ([Wiki](https://en.wikipedia.org/wiki/Comma-separated_values)) **Datensatz** ([Wiki](https://en.wikipedia.org/wiki/Data_set)) hochgeladen werden, bei dem die Zielvariable der **Klassifikation** ([Wiki](https://en.wikipedia.org/wiki/Classification)) in der ersten **Spalte** ([Wiki](https://en.wikipedia.org/wiki/Column_(database))) hinterlegt sein sollte und die dazugehörigen **Features** ([Wiki](https://en.wikipedia.org/wiki/Feature_(machine_learning))) entsprechend in den nachfolgenden Spalten. Der **IRIS** [Wiki](https://en.wikipedia.org/wiki/Iris_flower_data_set)) Datensatz im Ordner *data* ist ein geeignetes Klassifikationsbeispiel.

In [ ]:
# --- Erster Teil des Codes ---
import micropip
await micropip.install('ipywidgets')
import random, math
from IPython.display import display, clear_output, HTML
from ipywidgets import FileUpload
import ipywidgets as widgets

# Upload widget
uploader = FileUpload(accept='.csv', multiple=False)
display(uploader)

## 🧠 - Neuronales Netzwerk
Danach können die Anzahl an **Schichten** ([Wiki](https://en.wikipedia.org/wiki/Hidden_layer)) und **Neuronen** ([Wiki](https://en.wikipedia.org/wiki/Artificial_neuron)) - sowie die zugrundeliegenden **Aktivierungsfunktionen** ([Wiki](https://en.wikipedia.org/wiki/Activation_function)) - gemeinsam mit den **Epochen** ([Wiki](https://en.wikipedia.org/wiki/Learning_curve_(machine_learning))) und der **Lernrate** ([Wiki](https://en.wikipedia.org/wiki/Learning_rate)) des neuronalen Netzwerks spezifiziert werden.

In [ ]:
# --- (2) Hidden Code ---
def parse_csv_string(csv_string):
    lines = csv_string.strip().split('\n')
    y = []
    X = []
    for line in lines:
        parts = line.strip().split(',')
        if len(parts) < 2:
            continue
        y.append(int(parts[0]))
        x_row = [float(val) for val in parts[1:]]
        X.append(x_row)
    return X, y

# --- Split CSV ---
if uploader.value:
    uploaded_file = uploader.value[0]  # Corrected from .values()
    content = uploaded_file['content'].tobytes().decode('utf-8')
    X, y = parse_csv_string(content)

# --- Standardization ---
def normalize(X):
    transposed = list(zip(*X))
    mins = [min(col) for col in transposed]
    maxs = [max(col) for col in transposed]
    return [[(x_i - min_i) / (max_i - min_i + 1e-9) 
             for x_i, min_i, max_i in zip(x_row, mins, maxs)] for x_row in X]

X = normalize(X)

# --- Activation & Loss ---
def sigmoid(x): return 1 / (1 + math.exp(-x))
def sigmoid_derivative(out): return out * (1 - out)
def relu(x): return max(0, x)
def relu_derivative(out): return 1 if out > 0 else 0
def leaky_relu(x): return x if x > 0 else 0.01 * x
def leaky_relu_derivative(out): return 1 if out > 0 else 0.01
def tanh(x): return math.tanh(x)
def tanh_derivative(out): return 1 - out**2
def binary_cross_entropy(pred, y): 
    epsilon = 1e-7
    return - (y * math.log(pred + epsilon) + (1 - y) * math.log(1 - pred + epsilon))
def binary_cross_entropy_derivative(pred, y): 
    epsilon = 1e-7
    return -(y / (pred + epsilon)) + (1 - y) / (1 - pred + epsilon)

# --- Dense Layer ---
def dense_forward(x, w, b, act='relu'):
    pres = [sum(x[i] * w[j][i] for i in range(len(x))) + b[j] for j in range(len(w))]
    outs = []
    for z in pres:
        if act == 'sigmoid':
            outs.append(sigmoid(z))
        elif act == 'relu':
            outs.append(relu(z))
        elif act == 'leaky_relu':
            outs.append(leaky_relu(z))
        elif act == 'tanh':
            outs.append(tanh(z))
    return outs, pres

def dense_backward(x, grad_out, out, pre, w, b, act='relu', lr=0.01):
    grad_in = [0] * len(x)
    for j in range(len(w)):
        if act == 'sigmoid':
            delta = grad_out[j] * sigmoid_derivative(out[j])
        elif act == 'relu':
            delta = grad_out[j] * relu_derivative(pre[j])
        elif act == 'leaky_relu':
            delta = grad_out[j] * leaky_relu_derivative(pre[j])
        elif act == 'tanh':
            delta = grad_out[j] * tanh_derivative(out[j])
        for i in range(len(x)):
            grad_in[i] += w[j][i] * delta
            w[j][i] -= lr * delta * x[i]
        b[j] -= lr * delta
    return grad_in

# --- Visualisierung ---
def html_network_with_connections(layer_sizes, input_dim):
    all_layers = [input_dim] + layer_sizes + [1]
    neuron_size = 20
    margin = 10
    column_spacing = 80
    row_spacing = neuron_size + 20
    radius = neuron_size // 2
    total_width = len(all_layers) * column_spacing
    total_height = max(all_layers) * row_spacing + 2 * margin
    svg = f'<svg width="{total_width}" height="{total_height}" style="position:absolute; top:0; left:0;">'
    positions = []
    for li, n in enumerate(all_layers):
        layer_x = li * column_spacing + column_spacing // 2
        layer = []
        total_layer_height = (n - 1) * row_spacing
        offset_y = (total_height - total_layer_height) // 2
        for ni in range(n):
            layer_y = offset_y + ni * row_spacing
            layer.append((layer_x, layer_y))
        positions.append(layer)
    for i in range(len(positions)-1):
        for x1, y1 in positions[i]:
            for x2, y2 in positions[i+1]:
                svg += f'<line x1="{x1}" y1="{y1}" x2="{x2}" y2="{y2}" stroke="gray" stroke-width="1" />'
    svg += '</svg>'
    html = f'''
    <style>
        .network-wrapper {{
            position: relative;
            width: {total_width}px;
            height: {total_height}px;
        }}
        .network {{
            position: absolute;
            top: 0; left: 0;
            display: flex;
            flex-direction: row;
            justify-content: center;
            height: 100%;
        }}
        .layer {{
            display: flex;
            flex-direction: column;
            justify-content: center;
            align-items: center;
            width: {column_spacing}px;
        }}
        .neuron {{
            width: {neuron_size}px;
            height: {neuron_size}px;
            border: 2px solid black;
            border-radius: 50%;
            background-color: #f2f2f2;
            margin: 10px 0;
            box-sizing: border-box;
        }}
    </style>
    <div class="network-wrapper">
        {svg}
        <div class="network">
    '''
    for n_neurons in all_layers:
        html += '<div class="layer">'
        for _ in range(n_neurons):
            html += '<div class="neuron"></div>'
        html += '</div>'
    html += '</div></div>'
    display(HTML(html))

# --- Speicher ---
trained_model = {}

# --- Training ---
def train_model(X, y, layer_sizes, activations, epochs, lr):
    dims = [len(X[0])] + layer_sizes + [1]
    weights, biases = [], []
    for i in range(len(dims)-1):
        w = [[random.uniform(-0.5, 0.5) for _ in range(dims[i])] for _ in range(dims[i+1])]
        b = [random.uniform(-0.5, 0.5) for _ in range(dims[i+1])]
        weights.append(w)
        biases.append(b)
    loss_trace = []
    for epoch in range(epochs):
        total_loss = 0
        for xi, yi in zip(X, y):
            x = xi
            acts, pres = [], []
            for i in range(len(weights)):
                act = 'sigmoid' if i == len(weights)-1 else activations[i]
                x, pre = dense_forward(x, weights[i], biases[i], act)
                acts.append(x)
                pres.append(pre)
            loss = binary_cross_entropy(acts[-1][0], yi)
            total_loss += loss
            grad = [binary_cross_entropy_derivative(acts[-1][0], yi)]
            for i in reversed(range(len(weights))):
                act = 'sigmoid' if i == len(weights)-1 else activations[i]
                inp = xi if i == 0 else acts[i-1]
                grad = dense_backward(inp, grad, acts[i], pres[i], weights[i], biases[i], act, lr)
        loss_trace.append(total_loss)
    return {
        "weights": weights,
        "biases": biases,
        "loss_trace": loss_trace
    }

# --- Widgets ---
layers_slider = widgets.IntSlider(value=2, min=1, max=5, step=1, description='Schichten:')
neuron_and_activation_controls = widgets.VBox()
epochs_slider = widgets.IntSlider(value=100, min=10, max=500, step=10, description='Epochen:')
lr_slider = widgets.FloatSlider(value=0.05, min=0.001, max=1.0, step=0.01, description='Lernrate:')
train_button = widgets.Button(description="Trainieren")
train_output = widgets.Output()

def update_neuron_sliders(*args):
    count = layers_slider.value
    controls = []
    for i in range(count):
        neuron_slider = widgets.IntSlider(value=3, min=1, max=10, step=1, description=f'{i+1}. Schicht:')
        activation_dropdown = widgets.Dropdown(
            options=['sigmoid', 'relu', 'leaky_relu', 'tanh'],
            value='sigmoid',
            description='mit Funktion:'
        )
        controls.append(widgets.HBox([neuron_slider, activation_dropdown]))
    neuron_and_activation_controls.children = controls

layers_slider.observe(update_neuron_sliders, names='value')
update_neuron_sliders()

def on_train_click(b):
    train_output.clear_output()
    layer_sizes = []
    activations = []
    for control in neuron_and_activation_controls.children:
        neuron_slider, activation_dropdown = control.children
        layer_sizes.append(neuron_slider.value)
        activations.append(activation_dropdown.value)
    epochs = epochs_slider.value
    lr = lr_slider.value
    model = train_model(X, y, layer_sizes, activations, epochs, lr)
    trained_model.clear()
    trained_model.update({
        "weights": model["weights"],
        "biases": model["biases"],
        "loss_trace": model["loss_trace"],
        "layer_sizes": layer_sizes
    })
    with train_output:
        html_network_with_connections(layer_sizes, input_dim=len(X[0]))

train_button.on_click(on_train_click)

display(widgets.VBox([
    widgets.HTML("<b>1. Anzahl an Schichten (N)</b>"),
    layers_slider,
    widgets.HTML("<b>2. Anzahl an Neuronen (N) sowie Aktivierungsfunktionen</b>"),
    neuron_and_activation_controls,
    widgets.HTML("<b>3. Anzahl an Epochen (N)</b>"),
    epochs_slider,
    widgets.HTML("<b>4. Höhe der Lernrate (%)</b>"),
    lr_slider,
    train_button,
    train_output
]))

## 📊 - Evaluation
Zur abschließenden Evaluation des neuronalen Netzwerks wird der Verlauf der **Verlustfunktion** ([Wiki](https://en.wikipedia.org/wiki/Loss_functions_for_classification)) in den Epochen sowie eine **Confusion Matrix** ([Wiki](https://en.wikipedia.org/wiki/Confusion_matrix)) mit der dazugehörigen **Genauigkeit** ([Wiki](https://en.wikipedia.org/wiki/Accuracy_and_precision#In_classification)) in Prozent ausgewiesen. Sollten **False Positives** ([Wiki](https://en.wikipedia.org/wiki/False_positives_and_false_negatives#False_positive_error)) oder **False Negatives** ([Wiki](https://en.wikipedia.org/wiki/False_positives_and_false_negatives#False_negative_error)) vorliegen, so werden die entsprechenden **Reihen** ([Wiki](https://en.wikipedia.org/wiki/Row_(database))) für eine manuelle Sichtung zusätzlich ausgegeben. 

In [ ]:
# --- Dritter Teil des Codes ---
eval_button = widgets.Button(description="Bewerten")
eval_output = widgets.Output()

def predict(x, weights, biases):
    for i in range(len(weights)):
        act = 'sigmoid' if i == len(weights)-1 else 'relu'
        x, _ = dense_forward(x, weights[i], biases[i], act)
    return 1 if x[0] > 0.5 else 0

def on_eval_click(b):
    eval_output.clear_output()
    if not trained_model:
        with eval_output:
            print("Bitte zuerst das Modell trainieren.")
        return

    weights = trained_model["weights"]
    biases = trained_model["biases"]
    losses = trained_model["loss_trace"]

    ypred = []
    TP = TN = FP = FN = 0
    FP_indices = []
    FN_indices = []

    for idx, (true, xi) in enumerate(zip(y, X)):
        pred = predict(xi, weights, biases)
        ypred.append(pred)
        if true == pred:
            if true == 1: TP += 1
            else: TN += 1
        else:
            if true == 1:
                FN += 1
                FN_indices.append(idx)
            else:
                FP += 1
                FP_indices.append(idx)

    acc = (TP + TN) / len(y)

    with eval_output:
        print("Verlust (jede 10. Epoche):")
        for i in range(9, len(losses), 10):
            print(f"Epoche {i+1:>3}: Verlust = {losses[i]:.4f}")
        print("\nConfusion Matrix:")
        print(f"TN: {TN}  FP: {FP}")
        print(f"FN: {FN}  TP: {TP}")
        print(f"Genauigkeit: {acc:.2f}")
        print()
        print("\n❌ False Positives bei Index:", ', '.join(map(str, FP_indices)) if FP_indices else "Keine")
        print("❌ False Negatives bei Index:", ', '.join(map(str, FN_indices)) if FN_indices else "Keine")

eval_button.on_click(on_eval_click)
display(widgets.VBox([eval_button, eval_output]))


![Logo](https://github.com/statistical-thinking/ki-enna-demo/blob/main/content/img/statistical-thinking.png?raw=true)